In [1]:
import sys
import csv, re, copy
import pickle
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import *
import numpy as np
import time
import logging
from collections import Counter
from gensim.models import KeyedVectors
from pycorenlp import StanfordCoreNLP
from Dictionary import *
from nltk.tree import Tree
nlp = StanfordCoreNLP('http://localhost:9000')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


--------Successfully processed HGNCDictIndex--------
No. Entities: 42463
Index Information: Counter({1: 170542, 2: 2194, 3: 241, 4: 48, 5: 13, 6: 12, 7: 5, 8: 3, 1167: 1, 10: 1, 9: 1, 11: 1})
--------Successfully processed ChEBIDictIndex--------
No. Entities: 53905
Index Information: Counter({1: 221524, 2: 7348, 3: 508, 4: 97, 5: 21, 6: 8, 7: 5, 8: 1})
--------Successfully processed GOBPDictIndex--------
No. Entities: 30662
Index Information: Counter({1: 122846, 2: 583, 3: 91, 4: 14, 5: 4, 6: 3})
--------Successfully processed MESHDictIndex--------
No. Entities: 4782
Index Information: Counter({1: 53615})


In [2]:
def loadSentences(filename):
    f = open(filename, encoding="utf8")
    reader = csv.DictReader(f, delimiter='\t')
    sentences = [row for row in reader]
    return sentences

In [3]:
def loadAllTextSentences():
    sentences = loadSentences('dataset/Training.sentence')
    sentences.extend(loadSentences('dataset/SampleSet.sentence'))
    sentences.extend(loadSentences('dataset/Task1NeuV3_corrected.sentence'))
    # print(len(sentences))
    TextSentenceID = dict()
    vocabulary = set()
    for line in sentences:
        id = line['Sentence-ID'][4:]
        text = line['Sentence']
        output = nlp.annotate(text, properties={'annotators': 'tokenize', 'outputFormat': 'json'})
#         print('---------------------------------------------')
        if id not in TextSentenceID:
            TextSentenceID[id] = {'id': id,
                                  'text': text,
                                  'pmid': line['PMID'],
                                  'tokens': [i['originalText'] for i in output['tokens']]}
            vocabulary = vocabulary.union(set(TextSentenceID[id]['tokens']))
    #     else:
    #         assert (TextSentenceID[id]['text'] == line['Sentence']), 'ID: %s \n Text1: %s \n Text2: %s'%(id, TextSentenceID[id]['text'], line['Sentence']) 
    #         assert (TextSentenceID[id]['pmid'] == line['PMID']), 'ID: %s \n PMID1: %s \n PMID2: %s'%(id, TextSentenceID[id]['pmid'], line['PMID'])
    print('Download text sentences:', len(TextSentenceID), 'sentences')
    # print(TextSentenceID['10000072']) 
    # {'id': '10000072', 'text': 'it was found that a 6-fold increase in Fdft1 activity compared with that of the wild-type did not cause significant changes in HmgCoA reductase activity, while the amounts of synthesized dolichols and ergosterols increased by 80 and 32 percent respectively.', 'pmid': '10623644'}
    return TextSentenceID, vocabulary

In [4]:
# output = nlp.annotate('Hello World (This world!)', properties={'annotators': 'tokenize', 'outputFormat': 'json'})
# print(output)

In [4]:
def tokeniseBEL(bs, mapToHGNC = False):
    bs = re.subn(r',GOCCID:\d+', '', bs) # Replace GOCCID (additional parameters of tloc)
    bs = re.subn(r',sub\([\w,]*?\)', '', bs[0]) # Remove all sub(_,_,_)
    bs = re.subn(r',trunc\(\d+\)', '', bs[0]) # Remove all trunc(_)
    terms = re.findall(r'(a|bp|path|g|m|r|p)\(([A-Z]+)\:([^"]*?|".*?")(,pmod\(.*?\))?\)', bs[0])
    # print(terms)
    bs = re.subn(r'(a|bp|path|g|m|r|p)\(([A-Z]+)\:([^"]*?|".*?")(,pmod\(.*?\))?\)', '@', bs[0])
    assert bs[1] == len(terms)
    # print(bs)
    relations = re.findall(r'\s((?:->)|(?:-\|)|(?:increases)|(?:decreases)|(?:directlyIncreases)|(?:directlyDecreases))\s', bs[0])
    # print(relations)
    bs = re.subn(r'\s((?:->)|(?:-\|)|(?:increases)|(?:decreases)|(?:directlyIncreases)|(?:directlyDecreases))\s', '&', bs[0])
    assert bs[1] == len(relations)
    # print(bs)
    functions = re.findall(r'((?:act)|(?:complex)|(?:tloc)|(?:deg)|(?:kin)|(?:tscript)|(?:cat)|(?:sec)|(?:chap)|(?:gtp)|(?:pep)|(?:phos)|(?:ribo)|(?:tport)|(?:surf))', bs[0])
    # print(functions)
    bs = re.subn(r'((?:act)|(?:complex)|(?:tloc)|(?:deg)|(?:kin)|(?:tscript)|(?:cat)|(?:sec)|(?:chap)|(?:gtp)|(?:pep)|(?:phos)|(?:ribo)|(?:tport)|(?:surf))', '$', bs[0])
    assert bs[1] == len(functions)
    # print(bs[0]) # Term = @, Function = $, Relation = &
    bs = re.subn(r', ', ',', bs[0]) # Remove white space after comma
    template = bs[0]
    return stringToTokens(template, terms, relations, functions, mapToHGNC)

In [5]:
def stringToTokens(template, terms, relations, functions, mapToHGNC = False):
    aList = []
    typeDict = {'p':'p', 'bp':'bp', 'path':'path', 'a':'a', 'g':'p', 'm':'p', 'r':'p'}
    relationDict = {'->': ' -> ', '-|': ' -| ', 'increases':' -> ', 'decreases':' -| ', 'directlyIncreases':' -> ', 'directlyDecreases':' -| '}
    functionDict = {'act':'act', 'kin':'act', 'tscript':'act', 'cat':'act', 'chap':'act', 'gtp':'act', 'pep':'act', 'phos':'act', 'ribo':'act', 'tport':'act',
                    'complex':'complex', 
                    'tloc': 'tloc', 'sec':'tloc', 'surf':'tloc', 
                    'deg': 'deg'}
    for s in template:
        if s == '@': # Term
            termTuple = terms.pop(0)
            ns = termTuple[1]
            symbol = termTuple[2]
            if mapToHGNC:
                if ns == 'EGID': # Transform namespace (EGID to HGNC)
                    if symbol in EGID2HGNC:
                        ns = 'HGNC'
                        symbol = EGID2HGNC[symbol]
                        if any([a in symbol for a in [' ', '(', ')', '+', '-']]):
                            symbol = '"' + symbol + '"'
                    else:
                        return False
                elif ns == 'MGI': # Transform namespace (MGI to HGNC)
                    if symbol in MGI2HGNC:
                        ns = 'HGNC'
                        symbol = MGI2HGNC[symbol]
                        if any([a in symbol for a in [' ', '(', ')', '+', '-']]):
                            symbol = '"' + symbol + '"'
                    else:
                        return False
            if termTuple[3] == '':
                aList.extend([typeDict[termTuple[0]], '(', ns+':'+symbol, ')'])
            else:
                aList.extend([typeDict[termTuple[0]], '(', ns+':'+symbol, ',', 'pmod(P)', ')'])
        elif s == '&': # Relation
            aList.append(relationDict[relations.pop(0)])
        elif s == '$': # Function
            aList.append(functionDict[functions.pop(0)])
        else: # brackets and comma
            aList.append(s)
    return aList

In [6]:
TextSentenceID, vocabulary = loadAllTextSentences()
print(TextSentenceID['10000072'])
print(len(vocabulary))

Download text sentences: 6536 sentences
{'id': '10000072', 'text': 'it was found that a 6-fold increase in Fdft1 activity compared with that of the wild-type did not cause significant changes in HmgCoA reductase activity, while the amounts of synthesized dolichols and ergosterols increased by 80 and 32 percent respectively.', 'pmid': '10623644', 'tokens': ['it', 'was', 'found', 'that', 'a', '6-fold', 'increase', 'in', 'Fdft1', 'activity', 'compared', 'with', 'that', 'of', 'the', 'wild-type', 'did', 'not', 'cause', 'significant', 'changes', 'in', 'HmgCoA', 'reductase', 'activity', ',', 'while', 'the', 'amounts', 'of', 'synthesized', 'dolichols', 'and', 'ergosterols', 'increased', 'by', '80', 'and', '32', 'percent', 'respectively', '.']}
19049


In [7]:
word_vectors = KeyedVectors.load_word2vec_format('word_embeddings/PubMed-and-PMC-w2v.bin', binary=True)
print(word_vectors['Increases'])

[-1.25327557e-01 -2.05734119e-01  2.20678654e-02  1.27095148e-01
  4.70568202e-02  3.66582334e-01  1.80289820e-01 -6.96827620e-02
  5.25160849e-01  2.50934307e-02  1.86377347e-01 -1.57668844e-01
  5.11006951e-01  2.82196283e-01 -1.45905316e-01 -1.02183104e-01
 -1.58878171e-03 -2.69769728e-01  4.36125807e-02 -3.74512225e-02
  1.44765481e-01 -1.72953263e-01  5.64784929e-02  2.03118950e-01
 -2.29118302e-01 -3.89206707e-01  1.89598396e-01  8.48720893e-02
 -2.92850465e-01 -1.89046666e-01  3.03188503e-01 -4.85944226e-02
  2.32507274e-01  1.78006619e-01  9.79960859e-02  6.02323450e-02
  1.65033221e-01 -3.79372507e-01  1.18517898e-01 -1.47823170e-01
  1.21478774e-01 -2.50081658e-01  2.41490863e-02  1.28086820e-01
  3.87153685e-01 -1.73163749e-02 -1.84716210e-01 -2.07878187e-01
  9.35073644e-02  3.20446283e-01  6.42037690e-02 -4.05614406e-01
  7.49878660e-02 -1.26757715e-02  1.44438535e-01 -3.08646530e-01
  1.06738694e-02  2.82481462e-01 -2.62360632e-01 -2.80956089e-01
 -1.86210290e-01 -1.15877

In [8]:
emb_dim = 200
T_vocab_size = 50000
T_idx2w = ['_', 'unk'] + list(vocabulary) 
for word in word_vectors.vocab.keys():
    if word not in T_idx2w:
        T_idx2w.append(word)
    if len(T_idx2w) >= T_vocab_size + 2:
        break
T_idx2w.extend(['start_id', 'end_id'])
T_w2idx = dict([(T_idx2w[i], i) for i in range(len(T_idx2w))])
T_vocab_size_total = len(T_idx2w)

In [9]:
word_embedding = pickle.load(open("word_embedding.pickle", "rb"))

In [ ]:
word_embedding = np.random.uniform(-0.1, 0.1, (2, emb_dim))
count = 2
for i in range(2, T_vocab_size_total-2):
    if i%100 == 0:
        print(i)
    if T_idx2w[i] in word_vectors.vocab:
#         print(word_embedding.shape, np.array([word_vectors[T_idx2w[i]]]).shape)
        word_embedding = np.append(word_embedding, [word_vectors[T_idx2w[i]]], axis = 0)
    else:
        word_embedding = np.append(word_embedding, [np.random.uniform(-0.1, 0.1, emb_dim)], axis = 0)
        count += 1
word_embedding = np.append(word_embedding, [np.random.uniform(-0.1, 0.1, emb_dim)], axis = 0)
word_embedding = np.append(word_embedding, [np.random.uniform(-0.1, 0.1, emb_dim)], axis = 0)
print(word_embedding[T_w2idx['Increases']])
print(count+2)
pickle.dump(word_embedding, open("word_embedding.pickle", "wb"))

In [10]:
print(word_embedding.shape)

(50004, 200)


In [9]:
print(word_vectors['wild-type'])
print(word_vectors.vocab['wild-type'])
# for k in word_vectors.vocab.keys():
#     print(k)

[-0.03535113  0.03336161  0.1145009  -0.21139522  0.18536104  0.10084684
 -0.11079547  0.05809046  0.28275582  0.09770715  0.24691975  0.16967192
 -0.01673677  0.10089521 -0.1824921   0.14073035 -0.31860298 -0.41190234
 -0.13354068 -0.01240159 -0.18242343 -0.19114383  0.29078737 -0.17553619
  0.12014313  0.01050218  0.04176245 -0.2670518   0.20612988  0.00271289
  0.08800621  0.15186757 -0.10221786 -0.23187585  0.3860609   0.11735921
  0.41534626 -0.00043791 -0.25374904 -0.13475522 -0.17009191 -0.24757084
  0.2469786   0.30318278  0.03129166  0.02191979 -0.39349    -0.18328986
 -0.06172176 -0.20485361 -0.14117216  0.1620429   0.10013322  0.1703973
  0.0057947   0.03997155  0.01984005  0.31967682  0.03971784 -0.06181415
 -0.16398661  0.03618234 -0.2556029   0.06158364  0.06112619 -0.09901944
  0.04671043  0.02656536 -0.14078894 -0.30961394 -0.10270652  0.08510084
  0.17530367  0.19034943 -0.03502771  0.3173125   0.16265504  0.04707861
 -0.26934457 -0.09730205  0.1446976  -0.05900989 -0.

In [10]:
B_idx2w = ['_', 'unk', '(', ')', ',', 'p', 'a', 'bp', 'path', 'act', 'pmod(P)', 'tloc', 'complex', 'deg', ' -> ', ' -| ', ' -- ', 'PH:placeholder']
B_idx2w.extend(['HGNC:'+x if all([a not in x for a in [' ', '(', ')', '+', '-']]) else 'HGNC:"'+x+'"' for x in HGNCDict.keys()])
B_idx2w.extend(['CHEBI:'+x if all([a not in x for a in [' ', '(', ')', '+', '-']]) else 'CHEBI:"'+x+'"' for x in ChEBIDict.keys()])
B_idx2w.extend(['GOBP:'+x if all([a not in x for a in [' ', '(', ')', '+', '-']]) else 'GOBP:"'+x+'"' for x in GOBPDict.keys()])
B_idx2w.extend(['MESHD:'+x if all([a not in x for a in [' ', '(', ')', '+', '-']]) else 'MESHD:"'+x+'"' for x in MESHDict.keys()])
B_idx2w.extend(['start_id', 'end_id'])
B_w2idx = dict([(B_idx2w[i], i) for i in range(len(B_idx2w))])
B_vocab_size_total = len(B_idx2w)
print(B_idx2w[:200])
print(B_vocab_size_total)

['_', 'unk', '(', ')', ',', 'p', 'a', 'bp', 'path', 'act', 'pmod(P)', 'tloc', 'complex', 'deg', ' -> ', ' -| ', ' -- ', 'PH:placeholder', 'HGNC:A1BG', 'HGNC:"A1BG-AS1"', 'HGNC:A1CF', 'HGNC:A2M', 'HGNC:"A2M-AS1"', 'HGNC:A2ML1', 'HGNC:"A2ML1-AS1"', 'HGNC:"A2ML1-AS2"', 'HGNC:A2MP1', 'HGNC:A3GALT2', 'HGNC:A4GALT', 'HGNC:A4GNT', 'HGNC:A12M1', 'HGNC:A12M2', 'HGNC:A12M3', 'HGNC:A12M4', 'HGNC:AAAS', 'HGNC:AABT', 'HGNC:AACS', 'HGNC:AACSP1', 'HGNC:AADAC', 'HGNC:AADACL2', 'HGNC:"AADACL2-AS1"', 'HGNC:AADACL3', 'HGNC:AADACL4', 'HGNC:AADACP1', 'HGNC:AADAT', 'HGNC:AAED1', 'HGNC:AAGAB', 'HGNC:AAK1', 'HGNC:AAMDC', 'HGNC:AAMP', 'HGNC:AANAT', 'HGNC:AAR2', 'HGNC:AARD', 'HGNC:AARS', 'HGNC:AARS2', 'HGNC:AARSD1', 'HGNC:AARSP1', 'HGNC:AASDH', 'HGNC:AASDHPPT', 'HGNC:AASS', 'HGNC:AATBC', 'HGNC:AATF', 'HGNC:AATK', 'HGNC:AAVS1', 'HGNC:ABALON', 'HGNC:ABAT', 'HGNC:ABCA1', 'HGNC:ABCA2', 'HGNC:ABCA3', 'HGNC:ABCA4', 'HGNC:ABCA5', 'HGNC:ABCA6', 'HGNC:ABCA7', 'HGNC:ABCA8', 'HGNC:ABCA9', 'HGNC:"ABCA9-AS1"', 'HGNC:ABCA10'

In [ ]:
# sentences = loadSentences('dataset/TrainingNormalised.BEL')
# trainBSentences = [line['BEL-normalised'] for line in sentences]
# trainBTokenized = [tokeniseBEL(line) for line in trainBSentences]
# trainTSentences = [TextSentenceID[line['Sentence-ID'][4:]]['text'] for line in sentences]
# trainTTokenized = [TextSentenceID[line['Sentence-ID'][4:]]['tokens'] for line in sentences]
# assert len(trainBTokenized) == len(trainTTokenized)
# print(trainBTokenized[150], trainTTokenized[150])

In [11]:
def words2index(listOfWords, w2idx):
    lst = []
    for w in listOfWords:
        if w in w2idx:
            lst.append(w2idx[w])
        else:
            lst.append(w2idx['unk'])
#             print(listOfWords)
            if ' -> ' in w2idx:
                print('Word in BEL converted to unknown:', w)
            else:
                print('Word in Text converted to unknown:', w)
    return lst

In [ ]:
# trainT = [words2index(sublist, T_w2idx) for sublist in trainTTokenized]
# trainB = [words2index(sublist, B_w2idx) for sublist in trainBTokenized]
# print(trainB[150], trainT[150])

In [12]:
sentences = loadSentences('dataset/TrainingNormalised.BEL')
trainBSentences = []
trainBTokenized = []
trainTSentences = []
trainTTokenized = []
for line in sentences:
    BELTokens = tokeniseBEL(line['BEL-normalised'], mapToHGNC = True)
    if not BELTokens:
        print('Deleted BEL sentence:', line['BEL-normalised'])
        continue
    trainBSentences.append(line['BEL-normalised'])
    trainBTokenized.append(BELTokens)
    trainTSentences.append(TextSentenceID[line['Sentence-ID'][4:]]['text'])
    trainTTokenized.append(TextSentenceID[line['Sentence-ID'][4:]]['tokens'])
trainT = [words2index(sublist, T_w2idx) for sublist in trainTTokenized]
trainB = [words2index(sublist, B_w2idx) for sublist in trainBTokenized]
print(trainB[150], trainT[150])

Deleted BEL sentence: a(CHEBI:corticotropin) -> p(MGI:Sik1,pmod(P))
Deleted BEL sentence: act(p(EGID:12534)) -> p(HGNC:BTG1,pmod(P))
Deleted BEL sentence: act(p(EGID:279561)) -> p(HGNC:SLC12A4,pmod(P))
Deleted BEL sentence: act(p(EGID:279561)) -> p(HGNC:SLC12A4,pmod(P))
Deleted BEL sentence: act(p(MGI:Adrbk2)) -> p(HGNC:OPRK1,pmod(P))
Deleted BEL sentence: act(p(HGNC:SRPK1)) -> p(EGID:110809,pmod(P))
Deleted BEL sentence: act(p(HGNC:SRPK2)) -> p(EGID:110809,pmod(P))
Deleted BEL sentence: act(p(HGNC:TSSK6)) -> p(MGI:H3f3a,pmod(P))
Deleted BEL sentence: p(EGID:103968,pmod(P)) -> act(p(HGNC:PNPLA2))
Deleted BEL sentence: act(p(MGI:Ppp1cc)) -| p(HGNC:EIF2S1,pmod(P))
Deleted BEL sentence: p(MGI:Cdc42) -| p(HGNC:AXIN1,pmod(P))
Deleted BEL sentence: p(MGI:Cdc42) -> p(HGNC:GSK3B,pmod(P))
Deleted BEL sentence: p(MGI:Il3) -> p(HGNC:BCL2,pmod(P))
Deleted BEL sentence: p(MGI:Il3) -> p(HGNC:CRKL,pmod(P))
Deleted BEL sentence: p(MGI:Irs3) -| p(HGNC:IRS1,pmod(P))
Deleted BEL sentence: p(MGI:Irs3) -| 

In [ ]:
def model(encode_seqs, decode_seqs, is_train=True, reuse=False):
    with tf.variable_scope("model", reuse=reuse):
        # for translation, you need 2 seperated embedding layers
        with tf.variable_scope("embedding") as vs:
            net_encode = EmbeddingInputlayer(
                inputs = encode_seqs,
                vocabulary_size = T_vocab_size_total,
                embedding_size = emb_dim,
                name = 'encode_seq_embedding')
            net_decode = EmbeddingInputlayer(
                inputs = decode_seqs,
                vocabulary_size = B_vocab_size_total,
                embedding_size = emb_dim,
                name = 'decode_seq_embedding')
            vs.reuse_variables()
            tl.layers.set_name_reuse(True)
        net_rnn = Seq2Seq(net_encode, net_decode,
                cell_fn = tf.contrib.rnn.BasicLSTMCell,
                n_hidden = emb_dim,
                initializer = tf.random_uniform_initializer(-0.1, 0.1),
                encode_sequence_length = retrieve_seq_length_op2(encode_seqs),
                decode_sequence_length = retrieve_seq_length_op2(decode_seqs),
                initial_state_encode = None,
                dropout = (0.5 if is_train else None),
                n_layer = 1,
                return_seq_2d = True,
                name = 'seq2seq')
        net_out = DenseLayer(net_rnn, n_units=B_vocab_size_total, act=tf.identity, name='output')
    return net_out, net_rnn, net_encode

In [13]:
def modelWithAttention(encode_seqs, decode_seqs, is_train=True, reuse=False):
    with tf.variable_scope("model", reuse=reuse):
        # for translation, you need 2 seperated embedding layers
        with tf.variable_scope("embedding") as vs:
            net_encode = EmbeddingInputlayer(
                inputs = encode_seqs,
                vocabulary_size = T_vocab_size_total,
                embedding_size = emb_dim,
                name = 'encode_seq_embedding')
            net_decode = EmbeddingInputlayer(
                inputs = decode_seqs,
                vocabulary_size = B_vocab_size_total,
                embedding_size = emb_dim,
                name = 'decode_seq_embedding')
            vs.reuse_variables()
            tl.layers.set_name_reuse(True)
#         net_rnn = Seq2Seq(net_encode, net_decode,
#                 cell_fn = tf.contrib.rnn.BasicLSTMCell,
#                 n_hidden = emb_dim,
#                 initializer = tf.random_uniform_initializer(-0.1, 0.1),
#                 encode_sequence_length = retrieve_seq_length_op2(encode_seqs),
#                 decode_sequence_length = retrieve_seq_length_op2(decode_seqs),
#                 initial_state_encode = None,
#                 dropout = (0.5 if is_train else None),
#                 n_layer = 1,
#                 return_seq_2d = True,
#                 name = 'seq2seq')
#         with tf.variable_scope('seq2seq'):
#             tl.layers.set_name_reuse(True)
            # network = InputLayer(self.inputs, name=name+'/input')
        network_encode = DynamicRNNLayer(
            net_encode,
            cell_fn=tf.contrib.rnn.BasicLSTMCell,
            n_hidden=emb_dim,
            initializer=tf.random_uniform_initializer(-0.1, 0.1),
            initial_state=None,
            dropout=(0.5 if is_train else None),
            n_layer=1,
            sequence_length=retrieve_seq_length_op2(encode_seqs),
            return_last=False,
            return_seq_2d=False,
            name='seq2seq_encode')
#             attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
#                 num_units = emb_dim,
#                 memory = network_encode.outputs,
#                 memory_sequence_length = None # Might be made more accurate later 
#                 )
#             cell = tf.contrib.rnn.BasicLSTMCell(num_units = emb_dim)
#             attn_cell = tf.contrib.seq2seq.AttentionWrapper(cell, attention_mechanism)
#             network_decode = DynamicRNNLayer(
#                 net_decode,
#                 cell_fn=attn_cell,
#                 n_hidden=emb_dim,
#                 initializer=tf.random_uniform_initializer(-0.1, 0.1),
#                 initial_state=(network_encode.final_state if initial_state_decode is None else initial_state_decode),
#                 dropout=(0.5 if is_train else None),
#                 n_layer=1,
#                 sequence_length=retrieve_seq_length_op2(decode_seqs),
#                 return_last=False,
#                 return_seq_2d=True,
#                 name='seq2seq_decode')
        network_decode = DynamicRNNLayerWithAttention(
            net_decode,
            cell_fn=tf.contrib.rnn.BasicLSTMCell,
            attention_mechanism_fn=tf.contrib.seq2seq.BahdanauAttention,
            memory=network_encode.outputs,
            n_hidden=emb_dim,
            initializer=tf.random_uniform_initializer(-0.1, 0.1),
            initial_state=network_encode.final_state,
           # initial_state=(network_encode.final_state if is_train else None),
            dropout=(0.5 if is_train else None),
            n_layer=1,
            sequence_length=retrieve_seq_length_op2(decode_seqs),
            return_last=False,
            return_seq_2d=True,
            name='seq2seq_decode')
        net_out = DenseLayer(network_decode, n_units=B_vocab_size_total, act=tf.identity, name='output')
    return net_out, network_encode, net_encode, network_decode, network_decode.cell

In [14]:
class DynamicRNNLayerWithAttention(Layer):
    """
    The :class:`DynamicRNNLayer` class is a dynamic recurrent layer, see ``tf.nn.dynamic_rnn``.

    Parameters
    ----------
    layer : :class:`Layer`
        Previous layer
    cell_fn : TensorFlow cell function
        A TensorFlow core RNN cell
            - See `RNN Cells in TensorFlow <https://www.tensorflow.org/api_docs/python/>`__
            - Note TF1.0+ and TF1.0- are different
    cell_init_args : dictionary or None
        The arguments for the cell function.
    n_hidden : int
        The number of hidden units in the layer.
    initializer : initializer
        The initializer for initializing the parameters.
    sequence_length : tensor, array or None
        The sequence length of each row of input data, see ``Advanced Ops for Dynamic RNN``.
            - If None, it uses ``retrieve_seq_length_op`` to compute the sequence length, i.e. when the features of padding (on right hand side) are all zeros.
            - If using word embedding, you may need to compute the sequence length from the ID array (the integer features before word embedding) by using ``retrieve_seq_length_op2`` or ``retrieve_seq_length_op``.
            - You can also input an numpy array.
            - More details about TensorFlow dynamic RNN in `Wild-ML Blog <http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/>`__.
    initial_state : None or RNN State
        If None, `initial_state` is zero state.
    dropout : tuple of float or int
        The input and output keep probability (input_keep_prob, output_keep_prob).
            - If one int, input and output keep probability are the same.
    n_layer : int
        The number of RNN layers, default is 1.
    return_last : boolean or None
        Whether return last output or all outputs in each step.
            - If True, return the last output, "Sequence input and single output"
            - If False, return all outputs, "Synced sequence input and output"
            - In other word, if you want to stack more RNNs on this layer, set to False.
    return_seq_2d : boolean
        Only consider this argument when `return_last` is `False`
            - If True, return 2D Tensor [n_example, n_hidden], for stacking DenseLayer after it.
            - If False, return 3D Tensor [n_example/n_steps, n_steps, n_hidden], for stacking multiple RNN after it.
    dynamic_rnn_init_args : dictionary
        The arguments for ``tf.nn.dynamic_rnn``.
    name : str
        A unique layer name.

    Attributes
    ------------
    outputs : tensor
        The output of this layer.

    final_state : tensor or StateTuple
        The finial state of this layer.
            - When `state_is_tuple` is `False`, it is the final hidden and cell states, `states.get_shape() = [?, 2 * n_hidden]`.
            - When `state_is_tuple` is `True`, it stores two elements: `(c, h)`.
            - In practice, you can get the final state after each iteration during training, then feed it to the initial state of next iteration.

    initial_state : tensor or StateTuple
        The initial state of this layer.
            - In practice, you can set your state at the begining of each epoch or iteration according to your training procedure.

    batch_size : int or tensor
        It is an integer, if it is able to compute the `batch_size`; otherwise, tensor for dynamic batch size.

    sequence_length : a tensor or array
        The sequence lengths computed by Advanced Opt or the given sequence lengths, [batch_size]

    Notes
    -----
    Input dimension should be rank 3 : [batch_size, n_steps(max), n_features], if no, please see :class:`ReshapeLayer`.

    Examples
    --------
    Synced sequence input and output, for loss function see ``tl.cost.cross_entropy_seq_with_mask``.

    >>> input_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="input")
    >>> net = tl.layers.EmbeddingInputlayer(
    ...             inputs = input_seqs,
    ...             vocabulary_size = vocab_size,
    ...             embedding_size = embedding_size,
    ...             name = 'seq_embedding')
    >>> net = tl.layers.DynamicRNNLayer(net,
    ...             cell_fn = tf.contrib.rnn.BasicLSTMCell, # for TF0.2 use tf.nn.rnn_cell.BasicLSTMCell,
    ...             n_hidden = embedding_size,
    ...             dropout = (0.7 if is_train else None),
    ...             sequence_length = tl.layers.retrieve_seq_length_op2(input_seqs),
    ...             return_seq_2d = True,                   # stack denselayer or compute cost after it
    ...             name = 'dynamicrnn')
    ... net = tl.layers.DenseLayer(net, n_units=vocab_size, name="output")

    References
    ----------
    - `Wild-ML Blog <http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/>`__
    - `dynamic_rnn.ipynb <https://github.com/dennybritz/tf-rnn/blob/master/dynamic_rnn.ipynb>`__
    - `tf.nn.dynamic_rnn <https://github.com/tensorflow/tensorflow/blob/master/tensorflow/g3doc/api_docs/python/functions_and_classes/shard8/tf.nn.dynamic_rnn.md>`__
    - `tflearn rnn <https://github.com/tflearn/tflearn/blob/master/tflearn/layers/recurrent.py>`__
    - ``tutorial_dynamic_rnn.py``

    """

    def __init__(
            self,
            layer,
            cell_fn,  #tf.nn.rnn_cell.LSTMCell,
            attention_mechanism_fn,
            memory,
            cell_init_args=None,
            n_hidden=256,
            initializer=tf.random_uniform_initializer(-0.1, 0.1),
            sequence_length=None,
            initial_state=None,
            dropout=None,
            n_layer=1,
            return_last=None,
            return_seq_2d=False,
            dynamic_rnn_init_args=None,
            name='dyrnn',
    ):
#         self.initial_state_from_encoder = initial_state
        
        if dynamic_rnn_init_args is None:
            dynamic_rnn_init_args = {}
        if cell_init_args is None:
            cell_init_args = {'state_is_tuple': True}
        if return_last is None:
            return_last = True

        Layer.__init__(self, name=name)
        if cell_fn is None:
            raise Exception("Please put in cell_fn")
        if 'GRU' in cell_fn.__name__:
            try:
                cell_init_args.pop('state_is_tuple')
            except Exception:
                logging.warning("pop state_is_tuple fails.")
        self.inputs = layer.outputs
        self.memory = memory
        print("  [TL] DynamicRNNLayerWithAttention %s: n_hidden:%d, in_dim:%d in_shape:%s cell_fn:%s dropout:%s n_layer:%d" %
                     (self.name, n_hidden, self.inputs.get_shape().ndims, self.inputs.get_shape(), cell_fn.__name__, dropout, n_layer))

        logging.info("DynamicRNNLayerWithAttention %s: n_hidden:%d, in_dim:%d in_shape:%s cell_fn:%s dropout:%s n_layer:%d" %
                     (self.name, n_hidden, self.inputs.get_shape().ndims, self.inputs.get_shape(), cell_fn.__name__, dropout, n_layer))

        # Input dimension should be rank 3 [batch_size, n_steps(max), n_features]
        try:
            self.inputs.get_shape().with_rank(3)
        except Exception:
            raise Exception("RNN : Input dimension should be rank 3 : [batch_size, n_steps(max), n_features]")

        # Get the batch_size
        fixed_batch_size = self.inputs.get_shape().with_rank_at_least(1)[0]
        if fixed_batch_size.value:
            batch_size = fixed_batch_size.value
            logging.info("       batch_size (concurrent processes): %d" % batch_size)
        else:
            from tensorflow.python.ops import array_ops
            batch_size = array_ops.shape(self.inputs)[0]
            logging.info("       non specified batch_size, uses a tensor instead.")
        self.batch_size = batch_size

        # Creats the cell function
        # cell_instance_fn=lambda: cell_fn(num_units=n_hidden, **cell_init_args) # HanSheng
        rnn_creator = lambda: cell_fn(num_units=n_hidden, **cell_init_args)
        
        # ============================ PLJ added attention mechanism ====================================
        attention_mechanism = attention_mechanism_fn(num_units = n_hidden,
                                memory = self.memory,
                                memory_sequence_length = None # Might be made more accurate later 
                                )
        attn_cell_creator = lambda: tf.contrib.seq2seq.AttentionWrapper(rnn_creator(), attention_mechanism)
        # ===============================================================================================            
        # Apply dropout
        if dropout:
            if isinstance(dropout, (tuple, list)):
                in_keep_prob = dropout[0]
                out_keep_prob = dropout[1]
            elif isinstance(dropout, float):
                in_keep_prob, out_keep_prob = dropout, dropout
            else:
                raise Exception("Invalid dropout type (must be a 2-D tuple of " "float)")
            try:  # TF1.0
                DropoutWrapper_fn = tf.contrib.rnn.DropoutWrapper
            except Exception:
                DropoutWrapper_fn = tf.nn.rnn_cell.DropoutWrapper

            # cell_instance_fn1=cell_instance_fn        # HanSheng
            # cell_instance_fn=DropoutWrapper_fn(
            #                     cell_instance_fn1(),
            #                     input_keep_prob=in_keep_prob,
            #                     output_keep_prob=out_keep_prob)
            cell_creator = lambda is_last=True: \
                    DropoutWrapper_fn(attn_cell_creator(),
                                      input_keep_prob=in_keep_prob,
                                      output_keep_prob=out_keep_prob if is_last else 1.0)
        else:
            cell_creator = attn_cell_creator
        self.cell = cell_creator()
        # Apply multiple layers
        if n_layer > 1:
            try:
                MultiRNNCell_fn = tf.contrib.rnn.MultiRNNCell
            except Exception:
                MultiRNNCell_fn = tf.nn.rnn_cell.MultiRNNCell

            # cell_instance_fn2=cell_instance_fn # HanSheng
            try:
                # cell_instance_fn=lambda: MultiRNNCell_fn([cell_instance_fn2() for _ in range(n_layer)], state_is_tuple=True) # HanSheng
                self.cell = MultiRNNCell_fn([cell_creator(is_last=i == n_layer - 1) for i in range(n_layer)], state_is_tuple=True)
            except Exception:  # when GRU
                # cell_instance_fn=lambda: MultiRNNCell_fn([cell_instance_fn2() for _ in range(n_layer)]) # HanSheng
                self.cell = MultiRNNCell_fn([cell_creator(is_last=i == n_layer - 1) for i in range(n_layer)])

        # self.cell=cell_instance_fn() # HanSheng

        # Initialize initial_state
        if initial_state is None:
            self.initial_state = self.cell.zero_state(batch_size, dtype=D_TYPE)  # dtype=tf.float32)
        else:
            try:
                self.initial_state = self.cell.zero_state(batch_size, dtype=D_TYPE).clone(cell_state=initial_state) 
            except AttributeError:
                self.initial_state = initial_state

        # Computes sequence_length
        if sequence_length is None:
            try:  # TF1.0
                sequence_length = retrieve_seq_length_op(self.inputs if isinstance(self.inputs, tf.Tensor) else tf.stack(self.inputs))
            except Exception:  # TF0.12
                sequence_length = retrieve_seq_length_op(self.inputs if isinstance(self.inputs, tf.Tensor) else tf.pack(self.inputs))
        
        # Main - Computes outputs and last_states
#         with tf.variable_scope(tf.get_variable_scope()) as scope:
        with tf.variable_scope(name, initializer=initializer) as vs:
            outputs, last_states = tf.nn.dynamic_rnn(
                cell=self.cell,
                # inputs=X
                inputs=self.inputs,
                dtype=tf.float32,
                sequence_length=sequence_length,
                initial_state=self.initial_state,
                **dynamic_rnn_init_args)
            rnn_variables = tf.get_collection(TF_GRAPHKEYS_VARIABLES, scope=vs.name)

            # logging.info("     n_params : %d" % (len(rnn_variables)))
            # Manage the outputs
            if return_last:
                # [batch_size, n_hidden]
                # outputs = tf.transpose(tf.pack(outputs), [1, 0, 2]) # TF1.0 tf.pack --> tf.stack
                self.outputs = advanced_indexing_op(outputs, sequence_length)
            else:
                # [batch_size, n_step(max), n_hidden]
                # self.outputs = result[0]["outputs"]
                # self.outputs = outputs    # it is 3d, but it is a list
                if return_seq_2d:
                    # PTB tutorial:
                    # 2D Tensor [n_example, n_hidden]
                    try:  # TF1.0
                        self.outputs = tf.reshape(tf.concat(outputs, 1), [-1, n_hidden])
                    except Exception:  # TF0.12
                        self.outputs = tf.reshape(tf.concat(1, outputs), [-1, n_hidden])
                else:
                    # <akara>:
                    # 3D Tensor [batch_size, n_steps(max), n_hidden]
                    max_length = tf.shape(outputs)[1]
                    batch_size = tf.shape(outputs)[0]

                    try:  # TF1.0
                        self.outputs = tf.reshape(tf.concat(outputs, 1), [batch_size, max_length, n_hidden])
                    except Exception:  # TF0.12
                        self.outputs = tf.reshape(tf.concat(1, outputs), [batch_size, max_length, n_hidden])
                    # self.outputs = tf.reshape(tf.concat(1, outputs), [-1, max_length, n_hidden])
#             tf.get_variable_scope().reuse_variables()

        # Final state
        self.final_state = last_states

        self.sequence_length = sequence_length

        self.all_layers = list(layer.all_layers)
        self.all_params = list(layer.all_params)
        self.all_drop = dict(layer.all_drop)

        self.all_layers.extend([self.outputs])
        self.all_params.extend(rnn_variables)

In [15]:
batch_size = 16

In [ ]:
# # model for training
# encode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="encode_seqs") # encoding input  ['It', 'was', 'choking', 'with', 'smoke', '.', '_', '_']
# decode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="decode_seqs") # decoding input  ['start_id', 'Nó', 'đặc', 'khói', '.', '_']
# target_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_seqs") # decoding output ['Nó', 'đặc', 'khói', '.', 'end_id', '_']
# target_mask = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_mask") # tl.prepro.sequences_get_mask()
# net_out, _, net_encode = model(encode_seqs, decode_seqs, is_train=True, reuse=False)

# # model for inferencing
# encode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="encode_seqs")
# decode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="decode_seqs")
# net, net_rnn, _ = model(encode_seqs2, decode_seqs2, is_train=False, reuse=True)
# y = tf.nn.softmax(net.outputs)

In [16]:
g = tf.Graph()
with g.as_default() as graph:
    tl.layers.clear_layers_name()

    # model for training
    encode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="encode_seqs") # encoding input  ['It', 'was', 'choking', 'with', 'smoke', '.', '_', '_']
    decode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="decode_seqs") # decoding input  ['start_id', 'Nó', 'đặc', 'khói', '.', '_']
    target_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_seqs") # decoding output ['Nó', 'đặc', 'khói', '.', 'end_id', '_']
    target_mask = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_mask") # tl.prepro.sequences_get_mask()
    net_out, _, net_encode, _, _ = modelWithAttention(encode_seqs, decode_seqs, is_train=True, reuse=False)

    # model for inferencing
    encode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="encode_seqs")
    decode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="decode_seqs")
    net, net_rnn, _, network_decode, cell = modelWithAttention(encode_seqs2, decode_seqs2, is_train=False, reuse=True)
    y = tf.nn.softmax(net.outputs)

  [TL] EmbeddingInputlayer model/embedding/encode_seq_embedding: (50004, 200)
  [TL] EmbeddingInputlayer model/embedding/decode_seq_embedding: (131832, 200)


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


  [TL] DynamicRNNLayer model/seq2seq_encode: n_hidden:200, in_dim:3 in_shape:(16, ?, 200) cell_fn:BasicLSTMCell dropout:0.5 n_layer:1
       batch_size (concurrent processes): 16
  [TL] DynamicRNNLayerWithAttention model/seq2seq_decode: n_hidden:200, in_dim:3 in_shape:(16, ?, 200) cell_fn:BasicLSTMCell dropout:0.5 n_layer:1
  [TL] DenseLayer  model/output: 131832 identity
  [TL] EmbeddingInputlayer model/embedding/encode_seq_embedding: (50004, 200)
  [TL] EmbeddingInputlayer model/embedding/decode_seq_embedding: (131832, 200)
  [TL] DynamicRNNLayer model/seq2seq_encode: n_hidden:200, in_dim:3 in_shape:(1, ?, 200) cell_fn:BasicLSTMCell dropout:None n_layer:1
       batch_size (concurrent processes): 1
  [TL] DynamicRNNLayerWithAttention model/seq2seq_decode: n_hidden:200, in_dim:3 in_shape:(1, ?, 200) cell_fn:BasicLSTMCell dropout:None n_layer:1
  [TL] DenseLayer  model/output: 131832 identity


In [17]:
with g.as_default() as graph:
    loss = tl.cost.cross_entropy_seq_with_mask(logits=net_out.outputs, target_seqs=target_seqs, input_mask=target_mask, return_details=False, name='cost')
    net_out.print_params(False)
    lr = 0.0001
    train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

  param   0: model/embedding/decode_seq_embedding/embeddings:0 (131832, 200)      float32_ref
  param   1: model/seq2seq_decode/rnn/attention_wrapper/basic_lstm_cell/kernel:0 (600, 800)         float32_ref
  param   2: model/seq2seq_decode/rnn/attention_wrapper/basic_lstm_cell/bias:0 (800,)             float32_ref
  param   3: model/seq2seq_decode/rnn/attention_wrapper/bahdanau_attention/query_layer/kernel:0 (200, 200)         float32_ref
  param   4: model/seq2seq_decode/rnn/attention_wrapper/bahdanau_attention/attention_v:0 (200,)             float32_ref
  param   5: model/output/W:0     (200, 131832)      float32_ref
  param   6: model/output/b:0     (131832,)          float32_ref
  num of params: 53385632


In [18]:
with g.as_default() as graph:
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
    tl.layers.initialize_global_variables(sess)
    tl.files.assign_params(sess, [word_embedding], net_encode)
    tl.files.load_and_assign_npz(sess=sess, name='T2BWithAttention.npz', network=net)

[!] Load T2BWithAttention.npz failed!


In [ ]:
print(T_vocab_size_total)
print(word_embedding.shape)

In [25]:
with g.as_default() as graph:
    start_id = B_vocab_size_total-2
    end_id = B_vocab_size_total-1
    n_epoch = 50
    n_step = int(len(trainT)/batch_size)
    for epoch in range(n_epoch):
        epoch_time = time.time()
        ## shuffle training data
        from sklearn.utils import shuffle
        trainT, trainB = shuffle(trainT, trainB, random_state=0)
        ## train an epoch
        total_err, n_iter = 0, 0
        for X, Y in tl.iterate.minibatches(inputs=trainT, targets=trainB, batch_size=batch_size, shuffle=False):
            step_time = time.time()

            X = tl.prepro.pad_sequences(X)
            _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=end_id)
            _target_seqs = tl.prepro.pad_sequences(_target_seqs)

            _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=start_id, remove_last=False)
            _decode_seqs = tl.prepro.pad_sequences(_decode_seqs)
            _target_mask = tl.prepro.sequences_get_mask(_target_seqs)

            ## you can view the data here
    #         for i in range(len(X)):
    #             print(i, [E_idx2w[id] for id in X[i]])
    #             print(i, [V_idx2w[id] for id in _target_seqs[i]])
    #             print(i, [V_idx2w[id] for id in _decode_seqs[i]])
    #             print(i, _target_mask[i])
    #             print(len(_target_seqs[i]), len(_decode_seqs[i]), len(_target_mask[i]))
            # exit()

            _, err = sess.run([train_op, loss],
                            {encode_seqs: X,
                            decode_seqs: _decode_seqs,
                            target_seqs: _target_seqs,
                            target_mask: _target_mask})

            if n_iter % 10 == 0:
                print("Epoch[%d/%d] step:[%d/%d] loss:%f took:%.5fs" % (epoch, n_epoch, n_iter, n_step, err, time.time() - step_time))

            total_err += err; n_iter += 1

            ###============= inference
            if n_iter % 50 == 0:
                seeds = ["MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.",
                        "In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.",
                        "Thus, extramitochondrially targeted AIF is a dominant cell death inducer."] # p(MGI:Aifm1) increases bp(GOBP:"cell death")
                for seed in seeds:
                    print("Input >", seed)
                    output = nlp.annotate(seed, properties={'annotators': 'tokenize', 'outputFormat': 'json'})
                    seed_tokens = [i['originalText'] for i in output['tokens']]
                    seed_id = [T_w2idx[w] if w in T_w2idx else T_w2idx['unk'] for w in seed_tokens]

    #                 # 1. encode, get state
    #                 state = sess.run(net_rnn.final_state_encode,
    #                                 {encode_seqs2: [seed_id]})
    #                 # 2. decode, feed start_id, get first word
    #                 #   ref https://github.com/zsdonghao/tensorlayer/blob/master/example/tutorial_ptb_lstm_state_is_tuple.py
    #                 o, state = sess.run([y, net_rnn.final_state_decode],
    #                                 {net_rnn.initial_state_decode: state,
    #                                 decode_seqs2: [[start_id]]})
    #                 w_id = tl.nlp.sample_top(o[0], top_k=1)
    #                 w = B_idx2w[w_id]
    #                 # 3. decode, feed state iteratively
    #                 sentence = [w]
    #                 for _ in range(30): # max sentence length
    #                     o, state = sess.run([y, net_rnn.final_state_decode],
    #                                     {net_rnn.initial_state_decode: state,
    #                                     decode_seqs2: [[w_id]]})
    #                     w_id = tl.nlp.sample_top(o[0], top_k=1)
    #                     w = B_idx2w[w_id]
    #                     if w_id == end_id:
    #                         break
    #                     sentence = sentence + [w]
    #                 print(" >", ' '.join(sentence))

                    # 1. encode, get state
                    state, memory = sess.run([net_rnn.final_state, net_rnn.outputs],
                                    {encode_seqs2: [seed_id]})
                    # 2. decode, feed start_id, get first word
                    #   ref https://github.com/zsdonghao/tensorlayer/blob/master/example/tutorial_ptb_lstm_state_is_tuple.py
                    feed_dict = {network_decode.initial_state.cell_state:state,
                                 network_decode.memory:memory,
#                                     encode_seqs2: [seed_id],
                                    decode_seqs2: [[start_id]]}
#                     print(feed_dict)
                    o, state = sess.run([y, network_decode.final_state],
                                    feed_dict)
                    w_id = tl.nlp.sample_top(o[0], top_k=1)
                    w = B_idx2w[w_id]
                    # 3. decode, feed state iteratively
                    sentence = [w]
                    for _ in range(30): # max sentence length
                        o, state = sess.run([y, network_decode.final_state],
                                        {network_decode.initial_state:state,
                                         network_decode.memory:memory,
#                                          encode_seqs2: [seed_id],
                                        decode_seqs2: [[w_id]]})
                        w_id = tl.nlp.sample_top(o[0], top_k=1)
                        w = B_idx2w[w_id] ###
                        if w_id == end_id:
                            break
                        sentence = sentence + [w]
                    print(" >", ' '.join(sentence))

        print("Epoch[%d/%d] averaged loss:%f took:%.5fs" % (epoch, n_epoch, total_err/n_iter, time.time()-epoch_time))

        tl.files.save_npz(net.all_params, name='T2BWithAttention.npz', sess=sess)


Epoch[0/50] step:[0/657] loss:1.371322 took:2.49600s
Epoch[0/50] step:[10/657] loss:1.321491 took:2.55600s
Epoch[0/50] step:[20/657] loss:1.355456 took:3.53339s
Epoch[0/50] step:[30/657] loss:1.390600 took:2.87164s
Epoch[0/50] step:[40/657] loss:1.380458 took:3.06415s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( ) )  ->  act ( p ( , , , p ( pmod(P) ) ) , p ( , , ) ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( ) )  ->  act ( p ( , , ) ) )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > act ( p ( , , ) )  ->  p ( HGNC:STAT3 , pmod(P) )
Epoch[0/50] step:[50/657] loss:1.321382 took:2.77237s
Epoch[0/50] step:[60/657] loss:1.250979 took:3.28173s
Epoch[0/50] step:[70/657] loss:1.341184 took:3.19650s
Epoch[0/50] step:[80/657] loss:1.326299 took:2.56582s
Epoch[0/50] step:[90/657] loss:1.273731 took:2.72825s
Input > MMP-2 gelatinolyti

 > p ( HGNC:TNF )  ->  p ( HGNC:STAT3 , pmod(P) )
Epoch[0/50] step:[600/657] loss:1.382784 took:3.21956s
Epoch[0/50] step:[610/657] loss:1.219261 took:2.51268s
Epoch[0/50] step:[620/657] loss:1.265875 took:3.18848s
Epoch[0/50] step:[630/657] loss:1.258994 took:3.07919s
Epoch[0/50] step:[640/657] loss:1.329949 took:2.98694s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( ( , pmod(P) )  ->  p ( ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( HGNC:TNF )  ->  p ( ) )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > p ( HGNC:TNF )  ->  act ( p ( , , pmod(P) ) )
Epoch[0/50] step:[650/657] loss:1.271902 took:3.22758s
Epoch[0/50] averaged loss:1.315094 took:2020.03358s
[*] T2BWithAttention.npz saved
Epoch[1/50] step:[0/657] loss:1.309473 took:2.40540s
Epoch[1/50] step:[10/657] loss:1.222386 took:3.30479s
Epoch[1/50] step:[20/657] loss:1.2

 > p ( HGNC:TNF )  ->  p ( ) )
Epoch[1/50] step:[550/657] loss:1.324784 took:3.29075s
Epoch[1/50] step:[560/657] loss:1.226590 took:2.82050s
Epoch[1/50] step:[570/657] loss:1.364935 took:3.49329s
Epoch[1/50] step:[580/657] loss:1.236934 took:3.33587s
Epoch[1/50] step:[590/657] loss:1.206951 took:2.51769s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( ) )  ->  act ( p ( , , , p ( pmod(P) ) ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( HGNC:TNF )  ->  p ( HGNC:TNF )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > p ( HGNC:TNF )  ->  p ( HGNC:TNF )
Epoch[1/50] step:[600/657] loss:1.455637 took:3.23460s
Epoch[1/50] step:[610/657] loss:1.287495 took:3.12030s
Epoch[1/50] step:[620/657] loss:1.240101 took:2.61596s
Epoch[1/50] step:[630/657] loss:1.276527 took:3.22056s
Epoch[1/50] step:[640/657] loss:1.350706 took:2.47859s
Input > MMP

 > p ( HGNC:TNF )  ->  p ( ) )
Epoch[2/50] step:[500/657] loss:1.280144 took:3.07718s
Epoch[2/50] step:[510/657] loss:1.357517 took:3.13233s
Epoch[2/50] step:[520/657] loss:1.245011 took:2.90573s
Epoch[2/50] step:[530/657] loss:1.259344 took:2.80245s
Epoch[2/50] step:[540/657] loss:1.252403 took:3.17244s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( ( , pmod(P) )  ->  p ( ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( HGNC:TNF )  ->  p ( ) )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > p ( HGNC:TNF )  ->  act ( p ( , , , p , pmod(P) ) ) )
Epoch[2/50] step:[550/657] loss:1.422609 took:3.27972s
Epoch[2/50] step:[560/657] loss:1.365069 took:2.78841s
Epoch[2/50] step:[570/657] loss:1.374808 took:2.65506s
Epoch[2/50] step:[580/657] loss:1.338131 took:3.19550s
Epoch[2/50] step:[590/657] loss:1.364164 took:4.51400s
Input > MMP-2 g

 > p ( HGNC:TNF )  ->  p ( HGNC:STAT3 , pmod(P) )
Epoch[3/50] step:[450/657] loss:1.333986 took:3.10826s
Epoch[3/50] step:[460/657] loss:1.326752 took:3.79810s
Epoch[3/50] step:[470/657] loss:1.269817 took:3.24362s
Epoch[3/50] step:[480/657] loss:1.336277 took:3.60759s
Epoch[3/50] step:[490/657] loss:1.359444 took:3.54041s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( ( , pmod(P) )  ->  p ( ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( HGNC:TNF )  ->  p ( HGNC:TNF )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > act ( p ( pmod(P) ) )  ->  act ( p ( pmod(P) ) )
Epoch[3/50] step:[500/657] loss:1.333709 took:3.58353s
Epoch[3/50] step:[510/657] loss:1.234079 took:2.64704s
Epoch[3/50] step:[520/657] loss:1.231734 took:2.78641s
Epoch[3/50] step:[530/657] loss:1.372563 took:2.59189s
Epoch[3/50] step:[540/657] loss:1.349351 took:2.7

...

 > p ( ( , pmod(P) )  ->  act ( p ( ) ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( HGNC:TNF )  ->  p ( HGNC:TNF )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > act ( p ( pmod(P) ) )  ->  act ( p ( pmod(P) ) )
Epoch[14/50] step:[350/657] loss:1.169261 took:2.28608s
Epoch[14/50] step:[360/657] loss:1.080084 took:3.45619s
Epoch[14/50] step:[370/657] loss:1.246150 took:2.79343s
Epoch[14/50] step:[380/657] loss:1.236652 took:3.47223s
Epoch[14/50] step:[390/657] loss:1.146858 took:2.48661s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( ( , pmod(P) )  ->  act ( p ( HGNC:STAT3 ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( HGNC:TNF )  ->  p ( HGNC:TNF )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > act ( p ( pmod(P) ) )  ->

Epoch[15/50] step:[260/657] loss:1.136484 took:2.52822s
Epoch[15/50] step:[270/657] loss:1.161937 took:3.18146s
Epoch[15/50] step:[280/657] loss:1.150770 took:2.13919s
Epoch[15/50] step:[290/657] loss:1.330855 took:3.04109s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( ( , pmod(P) )  -|  act ( p ( HGNC:STAT3 ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( HGNC:TNF )  ->  p ( HGNC:TNF )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > act ( p ( pmod(P) ) )  ->  act ( p ( pmod(P) ) )
Epoch[15/50] step:[300/657] loss:1.296645 took:2.77188s
Epoch[15/50] step:[310/657] loss:1.210305 took:3.11879s
Epoch[15/50] step:[320/657] loss:1.209237 took:3.27471s
Epoch[15/50] step:[330/657] loss:1.273323 took:2.72225s
Epoch[15/50] step:[340/657] loss:1.240785 took:5.23744s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PB

 > p ( HGNC:TNF )  ->  p ( HGNC:TNF )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > act ( p ( pmod(P) ) )  ->  act ( p ( pmod(P) ) )
Epoch[16/50] step:[200/657] loss:1.393633 took:2.45553s
Epoch[16/50] step:[210/657] loss:1.355827 took:4.61728s
Epoch[16/50] step:[220/657] loss:1.326303 took:3.37598s
Epoch[16/50] step:[230/657] loss:1.147409 took:3.15539s
Epoch[16/50] step:[240/657] loss:1.216526 took:2.82150s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( ( , pmod(P) )  -|  act ( p ( HGNC:STAT3 ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( HGNC:TNF )  ->  p ( HGNC:TNF )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > act ( p ( pmod(P) ) )  ->  act ( p ( pmod(P) ) )
Epoch[16/50] step:[250/657] loss:1.144155 took:3.23861s
Epoch[16/50] step:[260/657] loss:1.166097 took:3.52738s
Epoch[16/50] 

Epoch[17/50] step:[130/657] loss:1.144520 took:2.90975s
Epoch[17/50] step:[140/657] loss:1.327051 took:2.79796s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( ( , pmod(P) )  ->  act ( p ( HGNC:STAT3 ) )
Input > In contrast, vandetanib treatment induced a 2.3-fold increase in eNOS mRNA in B16.F10 vasculature.
 > p ( HGNC:TNF )  ->  p ( HGNC:TGFB1 )
Input > Thus, extramitochondrially targeted AIF is a dominant cell death inducer.
 > act ( p ( pmod(P) ) )  ->  bp ( GOBP:"apoptotic process" )
Epoch[17/50] step:[150/657] loss:1.148600 took:2.44252s
Epoch[17/50] step:[160/657] loss:1.133721 took:3.39841s
Epoch[17/50] step:[170/657] loss:1.161965 took:3.10978s
Epoch[17/50] step:[180/657] loss:1.224728 took:3.32335s
Epoch[17/50] step:[190/657] loss:1.156701 took:3.01652s
Input > MMP-2 gelatinolytic activity was higher in cells infected with PBS (mock) and Ad-SV.
 > p ( HGNC:TNF )  ->  act ( p ( HGNC:MAPK1 ) )
Input > In contrast, vandetanib 

KeyboardInterrupt: 

In [23]:
from time import gmtime, strftime
sentences = loadSentences('dataset/Task1NeuV3_corrected.sentence')
sampleTSentences = [TextSentenceID[line['Sentence-ID'][4:]]['text'] for line in sentences]
sampleTTokenized = [TextSentenceID[line['Sentence-ID'][4:]]['tokens'] for line in sentences]
sampleT = [words2index(sublist, T_w2idx) for sublist in sampleTTokenized]
f = open('results/Task1NeuV3_corrected' + '_' + strftime("%Y%m%d%H%M%S", gmtime()) +'.txt', 'w')
for i in range(len(sampleT)):
    sentence_id = sentences[i]['Sentence-ID'][4:]
    seed_id = sampleT[i]
    # 1. encode, get state
    state = sess.run(net_rnn.final_state_encode,
                    {encode_seqs2: [seed_id]})
    # 2. decode, feed start_id, get first word
    #   ref https://github.com/zsdonghao/tensorlayer/blob/master/example/tutorial_ptb_lstm_state_is_tuple.py
    o, state = sess.run([y, net_rnn.final_state_decode],
                    {net_rnn.initial_state_decode: state,
                    decode_seqs2: [[start_id]]})
    w_id = tl.nlp.sample_top(o[0], top_k=1)
    w = B_idx2w[w_id]
    # 3. decode, feed state iteratively
    sentence = [w]
    for _ in range(30): # max sentence length
        o, state = sess.run([y, net_rnn.final_state_decode],
                        {net_rnn.initial_state_decode: state,
                        decode_seqs2: [[w_id]]})
        w_id = tl.nlp.sample_top(o[0], top_k=1)
        w = B_idx2w[w_id]
        if w_id == end_id:
            break
        sentence = sentence + [w]
    f.write(sentence_id + '\t' + ''.join(sentence) + '\n')
f.close()

In [6]:
sess = tf.InteractiveSession()

# sess.run(tf.initialize_all_variables())
tl.layers.initialize_global_variables(sess)

# tl.files.assign_params(sess, [load_params[0]], emb_net)
# print(sess.run(emb_net, {x:[1000, 1200]}))

emb_net.print_params()
emb_net.print_layers()

vector = sess.run(emb_net.outputs, feed_dict={x : [1000,122]})
print('vector:', vector.shape)

  param   0: embedding_layer/embeddings:0 (5000, 200)        float32_ref (mean: 8.860853995429352e-05, median: -7.748603820800781e-06, std: 0.05772264301776886)   
  num of params: 1000000
  layer   0: embedding_layer/embedding_lookup:0 <unknown>          float32
vector: (2, 200)
[[-0.0814091  -0.02825403 -0.05328007 -0.02780311 -0.0653923  -0.02430246
   0.01921391  0.09056108 -0.09929579 -0.04090867 -0.01154137  0.09985436
  -0.01916699 -0.00505853  0.04639273 -0.0912411   0.05427978 -0.09722042
  -0.00647257  0.08200861  0.02909157 -0.04913936 -0.02959277 -0.0851137
   0.00671704  0.09491635 -0.05910125  0.091788   -0.06434856  0.06943984
  -0.04222007  0.07136836 -0.01448791  0.08609817  0.00808146  0.03219343
   0.04411275  0.09080558 -0.03130624 -0.00718544  0.08492801  0.07755976
   0.00010915 -0.04611282 -0.00340497  0.07246631  0.03042834 -0.06478818
  -0.01676147 -0.09139509 -0.01971438  0.09120715 -0.07756324 -0.06961818
   0.03664833  0.02337506  0.09040568  0.00635266 -0.0

In [9]:
embed_pretrained = np.array([np.array([i for _ in range(200)]) for i in range(5000)])
tl.files.assign_params(sess, [embed_pretrained], emb_net)

[<tf.Tensor 'Assign:0' shape=(5000, 200) dtype=float32_ref>]

In [10]:
vector = sess.run(emb_net.outputs, feed_dict={x : [1000,122]})
print('vector:', vector.shape)
print(vector)

vector: (2, 200)
[[1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000. 1000.
  1000. 1000. 1000. 1